6 EMA strategy + leverage x3

In [ ]:
pip install python-binance

In [ ]:
pip install ta

<h1>Load yours data</h1>

In [15]:
import pandas as pd
from binance.client import Client
import ta
import matplotlib.pyplot as plt
import numpy as np

client = Client(tld='us')

pairName = "BTCUSDT"
startDate = "01 January 2020"
endDate = "15 June 2022"
timeInterval = Client.KLINE_INTERVAL_4HOUR

klinesT = client.get_historical_klines(pairName, timeInterval, startDate, endDate)

df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
df['close'] = pd.to_numeric(df['close'])
df['high'] = pd.to_numeric(df['high'])
df['low'] = pd.to_numeric(df['low'])
df['open'] = pd.to_numeric(df['open'])

df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']

print("Data loaded 100%")

Data loaded 100%


<h1>Define indicators</h1>

In [16]:
df.drop(df.columns.difference(['open','high','low','close','volume']), axis=1, inplace=True)

df['EMA1']= ta.trend.ema_indicator(close=df['close'], window=7)
df['EMA2']= ta.trend.ema_indicator(close=df['close'], window=30)
df['EMA3']= ta.trend.ema_indicator(close=df['close'], window=50)
df['EMA4']= ta.trend.ema_indicator(close=df['close'], window=100)
df['EMA5']= ta.trend.ema_indicator(close=df['close'], window=121)
df['EMA6']= ta.trend.ema_indicator(close=df['close'], window=200)

df['STOCH_RSI'] = ta.momentum.stochrsi(close=df['close'], window=14, smooth1=3, smooth2=3)

print("Indicators loaded 100%")

df

Indicators loaded 100%


,open,high,low,close,volume,EMA1,EMA2,EMA3,EMA4,EMA5,EMA6,STOCH_RSI
timestamp,,,,,,,,,,,,
2020-01-01 00:00:00,7180.46,7238.03,7175.62,7230.71,12.76044500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 04:00:00,7226.19,7231.70,7196.20,7205.50,1.11132000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 08:00:00,7209.50,7231.17,7181.17,7195.80,2.09735800,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 12:00:00,7193.32,7244.43,7193.32,7233.02,7.53150600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 16:00:00,7237.54,7245.54,7215.51,7223.72,5.49440100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-14 08:00:00,22884.58,22975.21,21730.11,21983.76,259.33136500,23078.418759,26317.895737,27489.075816,28722.392529,29065.955977,30328.218934,0.283488
2022-06-14 12:00:00,21992.23,22798.28,21815.90,22706.78,249.57867400,22985.509070,26084.920528,27301.534803,28603.271489,28961.707191,30252.383721,0.601474
2022-06-14 16:00:00,22688.66,22779.51,22122.41,22206.61,120.91975400,22790.784302,25834.706946,27101.733831,28476.604925,28850.967892,30172.326271,0.526314


<h1>RUN the BackTest</h1>

In [19]:
dfTest = df.copy()
# dfTest = df['2021-03-01':'2021-09-01']

dt = None
dt = pd.DataFrame(columns=['date', 'position', 'reason','price', 'frais', 'wallet', 'drawBack'])

leverage = 3
wallet = 1000
makerFee = 0.0002
takerFee = 0.0007

initalWallet = wallet
lastAth = wallet
previousRow = dfTest.iloc[0]
orderInProgress = ''
longIniPrice = 0
shortIniPrice = 0
longLiquidationPrice = 999999
shortLiquidationPrice = 0

def openLongCondition(row, previousRow):
    if (row['EMA1'] > row['EMA2']
    and row['EMA2'] > row['EMA3']
    and row['EMA3'] > row['EMA4']
    and row['EMA4'] > row['EMA5']
    and row['EMA5'] > row['EMA6']
    and row['STOCH_RSI'] < 0.70):
        return True
    else:
        return False

def closeLongCondition(row, previousRow):
    if row['EMA6'] > row['EMA1']:
        return True
    else:
        return False

def openShortCondition(row, previousRow):
    if ( row['EMA6'] > row['EMA5']
    and row['EMA5'] > row['EMA4']
    and row['EMA4'] > row['EMA3']
    and row['EMA3'] > row['EMA2']
    and row['EMA2'] > row['EMA1']
    and row['STOCH_RSI'] > 0.2 ):
        return True
    else:
        return False

def closeShortCondition(row, previousRow):
    if row['EMA1'] > row['EMA6']:
        return True
    else:
        return False

for index, row in dfTest.iterrows():

    if orderInProgress != '':

        if orderInProgress == 'LONG':

            if row['low'] < longLiquidationPrice:
                print('YOUR LONG HAVE BEEN LIQUIDATED the',index)
                break

            if closeLongCondition(row, previousRow) == True:
                orderInProgress = ''
                closePrice = row['close']
                closePriceWithFee = row['close'] - takerFee * row['close']
                pr_change = (closePriceWithFee - longIniPrice) / longIniPrice
                wallet = wallet + wallet*pr_change*leverage

                #print('Close LONG at',closePrice,"the", index, '| wallet :', wallet, '| result :', pr_change*100*leverage)

                if wallet > lastAth:
                    lastAth = wallet

                myrow = {'date': index, 'position': "LONG", 'reason': 'Close Long Market', 'price': closePrice,
                        'frais': takerFee * wallet * leverage, 'wallet': wallet, 'drawBack': (wallet-lastAth)/lastAth}
                myrow_df = pd.DataFrame([myrow])
                dt = pd.concat([dt, myrow_df], ignore_index=True)

        elif orderInProgress == 'SHORT':

            if row['high'] > shortLiquidationPrice:
                print('YOUR SHORT HAVE BEEN LIQUIDATED the',index)
                break

            if closeShortCondition(row, previousRow) == True:
                orderInProgress = ''
                closePrice = row['close']
                closePriceWithFee = row['close'] + takerFee * row['close']
                pr_change = -(closePriceWithFee - shortIniPrice) / shortIniPrice
                wallet = wallet + wallet*pr_change*leverage

                # print('Close SHORT at',closePrice,"the", index, '| wallet :', wallet, '| result :', pr_change*100*leverage)

                if wallet > lastAth:
                    lastAth = wallet

                myrow = {'date': index, 'position': "SHORT", 'reason': 'Close Short Market', 'price': closePrice,
                        'frais': takerFee * wallet * leverage, 'wallet': wallet, 'drawBack': (wallet-lastAth)/lastAth}
                myrow_df = pd.DataFrame([myrow])
                myrow_df = pd.DataFrame([myrow])
                dt = pd.concat([dt, myrow_df], ignore_index=True)

    if orderInProgress == '':

        if openLongCondition(row, previousRow) == True:
            orderInProgress = 'LONG'
            closePrice = row['close']
            longIniPrice = row['close'] + takerFee * row['close']
            tokenAmount = (wallet * leverage) / row['close']
            longLiquidationPrice = longIniPrice - (wallet/tokenAmount)

            # print('Open LONG at', closePrice, '$ the', index, '| Liquidation price :', longLiquidationPrice)

            myrow = {'date': index, 'position': "Open Long", 'reason': 'Open Long Market', 'price': closePrice,
                     'frais': takerFee * wallet * leverage, 'wallet': wallet, 'drawBack': (wallet-lastAth)/lastAth}
            myrow_df = pd.DataFrame([myrow])
            dt = pd.concat([dt, myrow_df], ignore_index=True)

        if openShortCondition(row, previousRow) == True:
            orderInProgress = 'SHORT'
            closePrice = row['close']
            shortIniPrice = row['close'] - takerFee * row['close']
            tokenAmount = (wallet * leverage) / row['close']
            shortLiquidationPrice = shortIniPrice + (wallet/tokenAmount)

            # print('Open SHORT', closePrice, '$ the', index, '| Liquidation price :', shortLiquidationPrice)

            myrow = {'date': index, 'position': "Open Short", 'reason': 'Open Short Market', 'price': closePrice,
                     'frais': takerFee * wallet * leverage, 'wallet': wallet, 'drawBack': (wallet-lastAth)/lastAth}
            myrow_df = pd.DataFrame([myrow])
            dt = pd.concat([dt, myrow_df], ignore_index=True)

dt = dt.set_index(dt['date'])
dt.index = pd.to_datetime(dt.index)
dt['resultat%'] = dt['wallet'].pct_change()*100

dt['tradeIs'] = ''
dt.loc[dt['resultat%'] > 0, 'tradeIs'] = 'Good'
dt.loc[dt['resultat%'] < 0, 'tradeIs'] = 'Bad'

iniClose = dfTest.iloc[0]['close']
lastClose = dfTest.iloc[len(dfTest)-1]['close']
holdPercentage = ((lastClose - iniClose)/iniClose)
holdWallet = holdPercentage * leverage * initalWallet
algoPercentage = ((wallet - initalWallet)/initalWallet)
vsHoldPercentage = ((wallet - holdWallet)/holdWallet) * 100

try:
    tradesPerformance = round(dt.loc[(dt['tradeIs'] == 'Good') | (dt['tradeIs'] == 'Bad'), 'resultat%'].sum() / dt.loc[(dt['tradeIs'] == 'Good') | (dt['tradeIs'] == 'Bad'), 'resultat%'].count(), 2)
except:
    tradesPerformance = 0
    print("There is no Good or Bad Trades in the BackTest.")

try:
    TotalGoodTrades = dt.groupby('tradeIs')['date'].nunique()['Good']
    AveragePercentagePositivTrades = round(dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].sum() / dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].count(), 2)
    idbest = dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].idxmax()
    bestTrade = str(round(dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].max(), 2))
except:
    TotalGoodTrades = 0
    AveragePercentagePositivTrades = 0
    idbest = ''
    bestTrade = 0
    print("There is no Good Trades in the BackTest.")

try:
    TotalBadTrades = dt.groupby('tradeIs')['date'].nunique()['Bad']
    AveragePercentageNegativTrades = round(dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].sum() / dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].count(), 2)
    idworst = dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].idxmin()
    worstTrade = round(dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].min(), 2)
except:
    TotalBadTrades = 0
    AveragePercentageNegativTrades = 0
    idworst = ''
    worstTrade = 0
    print("/!\ There is no Bad Trades in the BackTest..")

totalTrades = TotalBadTrades + TotalGoodTrades

try:
    TotalLongTrades = dt.groupby('position')['date'].nunique()['LONG']
    AverageLongTrades = round(dt.loc[dt['position'] == 'LONG', 'resultat%'].sum() / dt.loc[dt['position'] == 'LONG', 'resultat%'].count(), 2)
    idBestLong = dt.loc[dt['position'] == 'LONG', 'resultat%'].idxmax()
    bestLongTrade = str(round(dt.loc[dt['position'] == 'LONG', 'resultat%'].max(), 2))
    idWorstLong = dt.loc[dt['position'] == 'LONG', 'resultat%'].idxmin()
    worstLongTrade = str(round(dt.loc[dt['position'] == 'LONG', 'resultat%'].min(), 2))
except:
    AverageLongTrades = 0
    TotalLongTrades = 0
    bestLongTrade = ''
    idBestLong = ''
    idWorstLong = ''
    worstLongTrade = ''
    print("There is no LONG Trades in the BackTest.")

try:
    TotalShortTrades = dt.groupby('position')['date'].nunique()['SHORT']
    AverageShortTrades = round(dt.loc[dt['position'] == 'SHORT', 'resultat%'].sum() / dt.loc[dt['position'] == 'SHORT', 'resultat%'].count(), 2)
    idBestShort = dt.loc[dt['position'] == 'SHORT', 'resultat%'].idxmax()
    bestShortTrade = str(round(dt.loc[dt['position'] == 'SHORT', 'resultat%'].max(), 2))
    idWorstShort = dt.loc[dt['position'] == 'SHORT', 'resultat%'].idxmin()
    worstShortTrade = str(round(dt.loc[dt['position'] == 'SHORT', 'resultat%'].min(), 2))
except:
    AverageShortTrades = 0
    TotalShortTrades = 0
    bestShortTrade = ''
    idBestShort = ''
    idWorstShort = ''
    worstShortTrade = ''
    print("/!\ There is no SHORT Trades in the BackTest.")

try:
    totalGoodLongTrade = dt.groupby(['position', 'tradeIs']).size()['LONG']['Good']
except:
    totalGoodLongTrade = 0
    print("/!\ There is no good LONG Trades in the BackTest.")

try:
    totalBadLongTrade = dt.groupby(['position', 'tradeIs']).size()['LONG']['Bad']
except:
    totalBadLongTrade = 0
    print("/!\ There is no bad LONG Trades in the BackTest.")

try:
    totalGoodShortTrade = dt.groupby(['position', 'tradeIs']).size()['SHORT']['Good']
except:
    totalGoodShortTrade = 0
    print("/!\ There is no good SHORT Trades in the BackTest.")

try:
    totalBadShortTrade = dt.groupby(['position', 'tradeIs']).size()['SHORT']['Bad']
except:
    totalBadShortTrade = 0
    print("/!\ There is no bad SHORT Trades in the BackTest.")

TotalTrades = TotalGoodTrades + TotalBadTrades
winRateRatio = (TotalGoodTrades/TotalTrades) * 100

reasons = dt['reason'].unique()

print("BackTest finished, final wallet :",wallet,"$")
dt.head(20)


BackTest finished, final wallet : 5891.52067675458 $


C:\Users\antoi\AppData\Local\Temp\ipykernel_33924\413103268.py:120: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dt = pd.concat([dt, myrow_df], ignore_index=True)
C:\Users\antoi\AppData\Local\Temp\ipykernel_33924\413103268.py:138: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dt['resultat%'] = dt['wallet'].pct_change()*100


,date,position,reason,price,frais,wallet,drawBack,resultat%,tradeIs
date,,,,,,,,,
2020-02-03 04:00:00,2020-02-03 04:00:00,Open Long,Open Long Market,9376.39,2.100000,1000,0.000000,NaN,
2020-02-25 20:00:00,2020-02-25 20:00:00,LONG,Close Long Market,9319.74,2.053176,977.702983,-0.022297,-2.229702,Bad
2020-03-03 12:00:00,2020-03-03 12:00:00,Open Short,Open Short Market,8751.84,2.053176,977.702983,-0.022297,0.000000,
2020-04-06 08:00:00,2020-04-06 08:00:00,SHORT,Close Short Market,7105.09,3.205150,1526.261997,0.000000,56.106918,Good
2020-04-25 00:00:00,2020-04-25 00:00:00,Open Long,Open Long Market,7508.58,3.205150,1526.261997,0.000000,0.000000,
2020-06-15 04:00:00,2020-06-15 04:00:00,LONG,Close Long Market,9119.44,5.251671,2500.795896,0.000000,63.851023,Good
2020-06-23 20:00:00,2020-06-23 20:00:00,Open Long,Open Long Market,9613.86,5.251671,2500.795896,0.000000,0.000000,
2020-06-25 00:00:00,2020-06-25 00:00:00,LONG,Close Long Market,9099.63,4.388098,2089.570592,-0.164438,-16.443777,Bad
2020-07-02 20:00:00,2020-07-02 20:00:00,Open Short,Open Short Market,9085.99,4.388098,2089.570592,-0.164438,0.000000,


<h1>Print Complete BackTest Analyses</h1>

In [18]:
print("Pair Symbol :",pairName,)
print("Period : [" + str(dfTest.index[0]) + "] -> [" +
      str(dfTest.index[len(dfTest)-1]) + "]")
print("Starting balance :", initalWallet, "$")

print("\n- General Informations -")
print("Final balance :", round(wallet, 2), "$")
print("Performance vs US Dollar :", round(algoPercentage*100, 2), "%")
print("Buy and Hold Performence :", round(holdPercentage*100, 2),
      "% | with Leverage :", round(holdPercentage*100, 2)*leverage, "%")
print("Performance vs Buy and Hold :", round(vsHoldPercentage, 2), "%")
print("Best trade : +"+bestTrade, "%, the ", idbest)
print("Worst trade :", worstTrade, "%, the ", idworst)
print("Worst drawBack :", str(100*round(dt['drawBack'].min(), 2)), "%")
print("Total fees : ", round(dt['frais'].sum(), 2), "$")

print("\n- Trades Informations -")
print("Total trades on period :",totalTrades)
print("Number of positive trades :", TotalGoodTrades)
print("Number of negative trades : ", TotalBadTrades)
print("Trades win rate ratio :", round(winRateRatio, 2), '%')
print("Average trades performance :",tradesPerformance,"%")
print("Average positive trades :", AveragePercentagePositivTrades, "%")
print("Average negative trades :", AveragePercentageNegativTrades, "%")

print("\n- LONG Trades Informations -")
print("Number of LONG trades :",TotalLongTrades)
print("Average LONG trades performance :",AverageLongTrades, "%")
print("Best  LONG trade +"+bestLongTrade, "%, the ", idBestLong)
print("Worst LONG trade", worstLongTrade, "%, the ", idWorstLong)
print("Number of positive LONG trades :",totalGoodLongTrade)
print("Number of negative LONG trades :",totalBadLongTrade)
print("LONG trade win rate ratio :", round(totalGoodLongTrade/TotalLongTrades*100, 2), '%')

print("\n- SHORT Trades Informations -")
print("Number of SHORT trades :",TotalShortTrades)
print("Average SHORT trades performance :",AverageShortTrades, "%")
print("Best  SHORT trade +"+bestShortTrade, "%, the ", idBestShort)
print("Worst SHORT trade", worstShortTrade, "%, the ", idWorstShort)
print("Number of positive SHORT trades :",totalGoodShortTrade)
print("Number of negative SHORT trades :",totalBadShortTrade)
print("SHORT trade win rate ratio :", round(totalGoodShortTrade/TotalShortTrades*100, 2), '%')

print("\n- Trades Reasons -")
reasons = dt['reason'].unique()
for r in reasons:
    print(r+" number :", dt.groupby('reason')['date'].nunique()[r])

Pair Symbol : BTCUSDT
Period : [2020-01-01 00:00:00] -> [2022-06-15 00:00:00]
Starting balance : 1000 $

- General Informations -
Final balance : 5891.52 $
Performance vs US Dollar : 489.15 %
Buy and Hold Performence : 196.88 % | with Leverage : 590.64 %
Performance vs Buy and Hold : -0.25 %
Best trade : +538.61 %, the  2021-01-22 08:00:00
Worst trade : -49.36 %, the  2021-04-30 20:00:00
Worst drawBack : -71.0 %
Total fees :  796.94 $

- Trades Informations -
Total trades on period : 27
Number of positive trades : 11
Number of negative trades :  16
Trades win rate ratio : 40.74 %
Average trades performance : 26.22 %
Average positive trades : 94.5 %
Average negative trades : -20.72 %

- LONG Trades Informations -
Number of LONG trades : 14
Average LONG trades performance : 51.83 %
Best  LONG trade +538.61 %, the  2021-01-22 08:00:00
Worst LONG trade -22.3 %, the  2022-02-18 00:00:00
Number of positive LONG trades : 8
Number of negative LONG trades : 6
LONG trade win rate ratio : 57.14 %